### Save UM climate variables as a dataset

In [1]:
import shutil
import warnings
from datetime import datetime
from pathlib import Path

import iris
import numpy as np
from tqdm.notebook import tqdm as tqdm

In [2]:
from aeolus.coord import (
    interp_cubelist_from_height_to_pressure_levels,
    regrid_3d,
    replace_z_coord,
)
from aeolus.model import um

In [3]:
from util_commons import PLANETS, ROSE_SUITES

In [4]:
warnings.filterwarnings("ignore", module="iris")
warnings.filterwarnings("ignore", module="aeolus")

Dataset description

Title: Aerosol-free Met Office Unified Model simulations of the atmosphere of WASP-96b

Summary: This dataset contains the Met Office Unified Model output from aerosol-free simulations of the atmosphere of WASP-96b. The dataset includes 4 simulations: (1) 1x solar metallicity (mdh_0) with the chemical equilibrium scheme (equilibrium), (2) 1x solar metallicity with the chemical kinetics scheme (kinetics), (3) 10x solar metallicity (mdh_1) with the chemical equilibrium scheme, and (4) 10x solar metallicity with the chemical kinetics scheme. Each simulation was run for 1000 Earth days, except for the 10x solar metallicity kinetics simulation, which was run for 1500 Earth days, with instantaneous output produced every 10 days. Averages over the last 200 days are presented for the following variables: eastward wind (u), northward wind (v), upward air velocity (w), air pressure, air temperature and mole fractions of CH4, CO, CO2, H2O, HCN, NH3, H2 and He in air. All variables are presented either at a common height-latitude-longitude grid (on_hlevs) or interpolated from geometric height to log-spaced pressure levels covering 1e7-1e2 Pa and presented at a common pressure-latitude-longitude grid (on_plevs).

In [5]:
# Parameters
STASH = {  # UM local variable names : CF-compliant variable names
    "x_wind": "eastward_wind",
    "y_wind": "northward_wind",
    "upward_air_velocity": "upward_air_velocity",
    "air_pressure": "air_pressure",
    "air_temperature": "air_temperature",
    "m01s56i008": "mole_fraction_of_carbon_monoxide_in_air",
    "m01s56i009": "mole_fraction_of_water_vapour_in_air",
    "m01s56i012": "mole_fraction_of_hydrogen_cyanide_in_air",
    "m01s56i014": "mole_fraction_of_methane_in_air",
    "m01s56i019": "mole_fraction_of_ammonia_in_air",
    "m01s56i029": "mole_fraction_of_carbon_dioxide_in_air",
    "m01s56i007": "mole_fraction_of_molecular_hydrogen_in_air",
    "m01s56i030": "mole_fraction_of_helium_in_air",
}
global_attrs_hlev = {  # Global metadata for the data on a common height-latitude-longitude grid
    "title": "Aerosol-free Met Office Unified Model simulations of the atmosphere of WASP-96b.",
    "summary": "This dataset contains the Met Office Unified Model output from aerosol-free simulations of the atmosphere of WASP-96b. The dataset includes 4 simulations: (1) 1x solar metallicity (mdh_0) with the chemical equilibrium scheme (equilibrium), (2) 1x solar metallicity with the chemical kinetics scheme (kinetics), (3) 10x solar metallicity (mdh_1) with the chemical equilibrium scheme, and (4) 10x solar metallicity with the chemical kinetics scheme. Each simulation was run for 1000 Earth days, except for the 10x solar metallicity kinetics simulation, which was run for 1500 Earth days, with instantaneous output produced every 10 days. Averages over the last 200 days are presented for the following variables: eastward wind (u), northward wind (v), upward air velocity (w), air pressure, air temperature and mole fractions of CH4, CO, CO2, H2O, HCN, NH3, H2 and He in air. All variables are presented either at a common height-latitude-longitude grid (on_hlevs) or interpolated from geometric height to log-spaced pressure levels covering 1e7-1e2 Pa and presented at a common pressure-latitude-longitude grid (on_plevs).",
    "conventions": "CF-1.10,ACDD-1.3",
    "source": "Met Office Unified Model (UM)",
    "processing_level": "Averaged over the last 200 days.",
    "date_created": f"{datetime.utcnow():%Y-%m-%d %H:%M:%S} UTC",
    "creator_name": "Maria Zamyatina",
    "creator_email": "m.zamyatina@exeter.ac.uk",
    "creator_institution": "University of Exeter",
    "coverage_content_type": "modelResult",
}
global_attrs_plev = {  # Global metadata for the data on a common pressure-latitude-longitude grid
    "title": "Aerosol-free Met Office Unified Model simulations of the atmosphere of WASP-96b.",
    "summary": "This dataset contains the Met Office Unified Model output from aerosol-free simulations of the atmosphere of WASP-96b. The dataset includes 4 simulations: (1) 1x solar metallicity (mdh_0) with the chemical equilibrium scheme (equilibrium), (2) 1x solar metallicity with the chemical kinetics scheme (kinetics), (3) 10x solar metallicity (mdh_1) with the chemical equilibrium scheme, and (4) 10x solar metallicity with the chemical kinetics scheme. Each simulation was run for 1000 Earth days, except for the 10x solar metallicity kinetics simulation, which was run for 1500 Earth days, with instantaneous output produced every 10 days. Averages over the last 200 days are presented for the following variables: eastward wind (u), northward wind (v), upward air velocity (w), air pressure, air temperature and mole fractions of CH4, CO, CO2, H2O, HCN, NH3, H2 and He in air. All variables are presented either at a common height-latitude-longitude grid (on_hlevs) or interpolated from geometric height to log-spaced pressure levels covering 1e7-1e2 Pa and presented at a common pressure-latitude-longitude grid (on_plevs).",
    "conventions": "CF-1.10,ACDD-1.3",
    "source": "Met Office Unified Model (UM)",
    "processing_level": "Averaged over the last 200 days. Interpolated from geometric height to log-spaced pressure levels covering 1e7-1e2 Pa.",
    "date_created": f"{datetime.utcnow():%Y-%m-%d %H:%M:%S} UTC",
    "creator_name": "Maria Zamyatina",
    "creator_email": "m.zamyatina@exeter.ac.uk",
    "creator_institution": "University of Exeter",
    "coverage_content_type": "modelResult",
}

# IO
planet = "wasp96b"
tK = "tK_6"
PLANET = [planet]
path_to_dataset = (
    Path.home() / "datasets" / "papers" / "zamyatina2024_quenching_driven_depletion_and_asymmetries"
)

# Last 20 time slices
selected_fnames_dict = {}
for planet in PLANET:
    selected_fnames_dict[planet] = {}
    for exp in ["equilibrium", "kinetics"]:
        selected_fnames_dict[planet][exp] = {}
        for mdh in ["mdh_0", "mdh_1"]:
            selected_fnames_dict[planet][exp][mdh] = {}
            selected_fnames = []
            for ltr in ["b", "c", "f", "g", "h"]:  # [chr(i) for i in range(ord("b"), ord("i"))]
                if planet == "wasp96b" and mdh == "mdh_1" and exp == "kinetics":
                    for nmr in np.arange(1300, 1500, 10):
                        selected_fname = f"atmosa.p{ltr}000000{nmr}_00"
                        selected_fnames.append(selected_fname)
                else:
                    for nmr in np.arange(800, 1000, 10):
                        selected_fname = f"atmosa.p{ltr}000000{nmr}_00"
                        selected_fnames.append(selected_fname)
                selected_fnames_dict[planet][exp][mdh] = {"selected_fnames": selected_fnames}

# Target pressure levels
tgt_plev = np.logspace(7, 2, 65)

# For cleaning up metadata
k2pop = ["initial_pt_profile_file", "star_spectrum_lw", "star_spectrum_sw", "dir_for_data"]

In [6]:
# !Choose mdh; one mdh per notebook run, takes time!
mdh = "mdh_1"

vrbls = {}
for planet in PLANET:
    vrbls[planet] = {}
    for exp in ["equilibrium", "kinetics"]:
        vrbls[planet][exp] = {}
        flist = []

        # Find data
        path_to_sim = ROSE_SUITES[planet][exp][mdh][tK]["dir_for_data"] / "raw"
        for f in selected_fnames_dict[planet][exp][mdh]["selected_fnames"]:
            flist.append(path_to_sim / f)

        # Select variables
        cl = iris.load(flist, STASH.keys())

        # Make variable names CF-compliant
        for cb in cl:
            for k, v in STASH.items():
                if k == cb.name():
                    cb.rename(v)
                    if "mole_fraction" in v:
                        cb.units = "1"

        # Regrid all onto a common height-latitude-longitude grid
        cl_hlev = iris.cube.CubeList()
        for cb in cl:
            # Replace model_level_number by level_height as a dimensional vertical coordinate
            temp_hlev = replace_z_coord(cl.extract_cube(um.temp))
            # Regrid in the horizontal and in the vertical onto coordinates of air_temperature
            new_cb = regrid_3d(replace_z_coord(cb, model=um), temp_hlev, model=um)
            cl_hlev.append(new_cb)

        # Regrid all onto a common pressure-latitude-longitude grid
        cl_plev = interp_cubelist_from_height_to_pressure_levels(cl_hlev, tgt_plev)

        # Delete unnecessary local metadata
        local_attrs = ROSE_SUITES[planet][exp][mdh][tK]
        for k in k2pop:
            local_attrs.pop(k, None)
        local_attrs["planet"] = PLANETS[planet]["tex"]
        local_attrs["mdh"] = mdh[-1]
        local_attrs["initial_pt_profile_info"] = ROSE_SUITES[planet][exp][mdh][
            "initial_pt_profile_info"
        ]

        cl_hlev_mean = iris.cube.CubeList()
        cl_plev_mean = iris.cube.CubeList()
        for cb_hlev in cl_hlev:
            # Average over the last 200 days
            cb_hlev_mean = cb_hlev.collapsed("time", iris.analysis.MEAN)
            # Clean up metadata
            cb_hlev_mean.remove_coord("forecast_period")
            cb_hlev_mean.remove_coord("forecast_reference_time")
            del cb_hlev_mean.attributes["STASH"]
            del cb_hlev_mean.attributes["source"]  # replaced by an entry from global_attrs
            cb_hlev_mean.attributes.update({**global_attrs_hlev, **local_attrs})
            cl_hlev_mean.append(cb_hlev_mean)
        for cb_plev in cl_plev:
            cb_plev_mean = cb_plev.collapsed("time", iris.analysis.MEAN)
            cb_plev_mean.remove_coord("forecast_period")
            cb_plev_mean.remove_coord("forecast_reference_time")
            del cb_plev_mean.attributes["STASH"]
            del cb_plev_mean.attributes["source"]
            cb_plev_mean.attributes.update({**global_attrs_plev, **local_attrs})
            cl_plev_mean.append(cb_plev_mean)

        # Assemble data
        vrbls[planet][exp] = {"cl_hlev_mean": cl_hlev_mean, "cl_plev_mean": cl_plev_mean}

In [7]:
cl_hlev_mean

[<iris 'Cube' of eastward_wind / (m s-1) (level_height: 66; latitude: 90; longitude: 144)>,
<iris 'Cube' of northward_wind / (m s-1) (level_height: 66; latitude: 90; longitude: 144)>,
<iris 'Cube' of upward_air_velocity / (m s-1) (level_height: 66; latitude: 90; longitude: 144)>,
<iris 'Cube' of air_pressure / (Pa) (level_height: 66; latitude: 90; longitude: 144)>,
<iris 'Cube' of air_temperature / (K) (level_height: 66; latitude: 90; longitude: 144)>,
<iris 'Cube' of mole_fraction_of_carbon_monoxide_in_air / (1) (level_height: 66; latitude: 90; longitude: 144)>,
<iris 'Cube' of mole_fraction_of_water_vapour_in_air / (1) (level_height: 66; latitude: 90; longitude: 144)>,
<iris 'Cube' of mole_fraction_of_hydrogen_cyanide_in_air / (1) (level_height: 66; latitude: 90; longitude: 144)>,
<iris 'Cube' of mole_fraction_of_methane_in_air / (1) (level_height: 66; latitude: 90; longitude: 144)>,
<iris 'Cube' of mole_fraction_of_ammonia_in_air / (1) (level_height: 66; latitude: 90; longitude: 144)>,
<iris 'Cube' of mole_fraction_of_carbon_dioxide_in_air / (1) (level_height: 66; latitude: 90; longitude: 144)>,
<iris 'Cube' of mole_fraction_of_molecular_hydrogen_in_air / (1) (level_height: 66; latitude: 90; longitude: 144)>,
<iris 'Cube' of mole_fraction_of_helium_in_air / (1) (level_height: 66; latitude: 90; longitude: 144)>]

In [8]:
cl_plev_mean

[<iris 'Cube' of eastward_wind / (m s-1) (air_pressure: 65; latitude: 90; longitude: 144)>,
<iris 'Cube' of northward_wind / (m s-1) (air_pressure: 65; latitude: 90; longitude: 144)>,
<iris 'Cube' of upward_air_velocity / (m s-1) (air_pressure: 65; latitude: 90; longitude: 144)>,
<iris 'Cube' of air_temperature / (K) (air_pressure: 65; latitude: 90; longitude: 144)>,
<iris 'Cube' of mole_fraction_of_carbon_monoxide_in_air / (1) (air_pressure: 65; latitude: 90; longitude: 144)>,
<iris 'Cube' of mole_fraction_of_water_vapour_in_air / (1) (air_pressure: 65; latitude: 90; longitude: 144)>,
<iris 'Cube' of mole_fraction_of_hydrogen_cyanide_in_air / (1) (air_pressure: 65; latitude: 90; longitude: 144)>,
<iris 'Cube' of mole_fraction_of_methane_in_air / (1) (air_pressure: 65; latitude: 90; longitude: 144)>,
<iris 'Cube' of mole_fraction_of_ammonia_in_air / (1) (air_pressure: 65; latitude: 90; longitude: 144)>,
<iris 'Cube' of mole_fraction_of_carbon_dioxide_in_air / (1) (air_pressure: 65; latitude: 90; longitude: 144)>,
<iris 'Cube' of mole_fraction_of_molecular_hydrogen_in_air / (1) (air_pressure: 65; latitude: 90; longitude: 144)>,
<iris 'Cube' of mole_fraction_of_helium_in_air / (1) (air_pressure: 65; latitude: 90; longitude: 144)>]

In [9]:
for planet in PLANET:
    for exp in ["equilibrium", "kinetics"]:
        fname_hlev = f"met_office_um_climate_variables_{planet}_{exp}_mdh{mdh[-1]}_on_hlevs.nc"
        fname_plev = f"met_office_um_climate_variables_{planet}_{exp}_mdh{mdh[-1]}_on_plevs.nc"
        iris.save(
            vrbls[planet][exp]["cl_hlev_mean"],
            str(path_to_dataset / "met_office_um_climate_variables" / fname_hlev),
        )
        iris.save(
            vrbls[planet][exp]["cl_plev_mean"],
            str(path_to_dataset / "met_office_um_climate_variables" / fname_plev),
        )

In [6]:
# Zip dataset
shutil.make_archive(
    str(path_to_dataset / "met_office_um_climate_variables"),
    "zip",
    str(path_to_dataset / "met_office_um_climate_variables"),
);